Team Members:
- Emily Liang 79453973
- Kristen Chung 42617410
- Kaomi Booker 85786904 
- Angie Xetey 44067973

In [98]:
# Install required packages
%pip install gradio unstructured sentence-transformers
%pip install google.generativeai     # for using local IDE
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Note: you may need to restart the kernel to use updated packages.


In [99]:
import sys
# print(sys.executable)
!{sys.executable} -m pip install google-generativeai

In [ ]:
# Import necessary libraries
import os
import time
from PIL import Image
import google.generativeai as genai
import gradio as gr 

In [ ]:
# Import types module from google.generativeai & API key setup
from google.generativeai import types 
import os

api_key = os.getenv("GOOGLE_API_KEY")
if api_key is None: # Check if API key exists
    raise ValueError("API key not found. Please set GOOGLE_API_KEY in your environment.")
genai.configure(api_key=api_key)
model = genai.GenerativeModel(model_name="gemini-2.5-flash-lite")

In [ ]:
# PART 1: Transcribing handwritten Python code into a code snippet - Emily

import re

# Goal: take an img including handwritten python code, feed it into LLM, & return the python code as text
def transcribe_image_code(image_file: Image.Image) -> str:
    """
    Step 1: Transcribes code from the input image using a multimodal LLM.

    Parameters:
        image_file (Image.Image): The uploaded image containing handwritten code.

    Returns:
        str: The transcribed code as a string.
    """
    
    # prompt given to the LLM to transcribe the code from the image
    prompt = """Given the following image containing handwritten Python code, transcribe the code accurately into a text format and output it. 
    Do NOT change the function's signature."""

    # call the LLM with the prompt and image
    output = model.generate_content([prompt, image_file])

    # extract the transcribed code from the LLM output & ensure it has text
    code = output.text if output and output.text else ""

    # remove any markdown formatting (like ```python ... ```) from the transcribed code
    code = re.sub(r"```(?:python)?\n?", "", code) 
    code = code.replace("```", "")

    # return the cleaned code as a string
    return code.strip()

# Testing the function with an example image
img_path = "code_images_example/code_image_01.jpg"
image = Image.open(img_path)
print(transcribe_image_code(image))


def bucketsort(arr, k):
    counts = [0] * k
    for x in arr:
        counts[x] += 1
    sorted_arr = []
    for i, count in enumerate(counts):
        sorted_arr.extend([i] * count)
    return sorted_arr


In [103]:
# PART 2: Running static analysis and explaining bugs (if any) in natural language - Kristen
import json
import re
import gradio as gr
import google.generativeai as genai
import unicodedata

def analyze_code(code_block: str):
    """
    Step 2: Performs static analysis on the transcribed code.

    Parameters:
        code_block (str): The transcribed code snippet.

    Returns:
        (1) The analysis text explaining bugs, improvements, or efficiency suggestions.
        (2) The refined version of the code snippet with the suggested fixes and enhancements.
    """
    # if not a valid code snippet, raise error
    if not code_block:
        raise gr.Error("Please provide a valid code snippet.")
    
    # get the model to analyze the code
    try:
        model = genai.GenerativeModel(model_name="gemini-2.5-flash-lite")
        
        # craft the prompt for code analysis
        prompt = f"""
        You are an expert Python developer and code reviewer.
        Analyze the following code snippet and do the following:
        1. Explain what the code does in natural language.
        2. Identify any syntax or logical bugs.
        3. Suggest specific fixes and efficiency improvements.
        4. Provide a refined version of the code with your fixes applied.

        IMPORTANT: Keep the original function name exactly as it is. Do not rename or change it. Only suggest fixes or efficiency improvements.

        Output your response in the following JSON format:
        {{
            "analysis": "detailed explanation and improvements",
            "refined_code": "refined Python code"
        }}

        Code to analyze:
        ```
        {code_block}
        ```
        """
        # get the model response
        try:
            response = model.generate_content(prompt)
            text = (response.text or "").strip()
        except Exception as e:
            raise gr.Error(f"Error generating content: {e}")

        text = text.replace("```json", "").replace("```", "").replace("python","").strip()

        # parse the JSON response
        try:
            data = json.loads(text)
        except json.JSONDecodeError:
            match = re.search(r"\{.*\}", text, re.DOTALL)
            if match:
                json_text = match.group(0)
                try:
                    data = json.loads(json_text)
                except json.JSONDecodeError:
                    # still fails → return raw text
                    data = {"analysis": text, "refined_code": code_block}
            else:
                # no JSON found → return raw text
                data = {"analysis": text, "refined_code": code_block}

        # extract analysis and refined code
        analysis = data.get("analysis", "No analysis found.")
        refined = data.get("refined_code", code_block)

        return analysis, refined

    except Exception as e:
        raise gr.Error(f"Error during analysis: {e}")

# test call
img_path = "code_images_example/code_image_01.jpg"
image = Image.open(img_path)
transcribed_code = transcribe_image_code(image)
analysis, refined = analyze_code(transcribed_code)
print(analysis)
print(refined)

The provided Python code defines a function called `bucketsort` that attempts to implement a counting sort algorithm. It takes two arguments: `arr`, which is expected to be a list of non-negative integers, and `k`, which represents the maximum possible value in `arr` plus one (i.e., the range of values is from 0 to k-1). 

The function first initializes a `counts` list of size `k` with all elements set to 0. It then iterates through the input array `arr`, and for each element `x`, it increments the count at index `x` in the `counts` list. This step correctly populates the `counts` list, where `counts[i]` will store the number of occurrences of the value `i` in `arr`.

However, there is a logical bug in the second loop. It iterates through `arr` again, but uses `enumerate(arr)` which yields `(index, value)` pairs. The code then uses `i` (which is the index of `arr`, not the value) to extend the `sorted_arr` by `count` times. This means it will be adding the index of the original array e

In [ ]:
# PART 3: Suggesting bug fixes, improvements, or efficiency tweaks to the code snippet - Kaomi

import json
import importlib.util
from inspect import isgenerator

# Map function names to their test JSON files
FUNCTION_TEST_FILES = {
    "bucketsort": "test_case_bucketsort.json",
    "flatten": "test_case_flatten.json",
    "find_in_sorted": "test_case_find_in_sorted.json",
    "pascal": "test_case_pascal.json",
    "possible_change": "test_case_possible_change.json"
}

def run_tests(filename_original, function_name, json_test_path):
    """Run test cases on a function from a Python file."""
    # open the test cases from the JSON file
    with open(json_test_path, 'r') as f:
        test_cases = json.load(f)["test_case"]

    # dynamically import the function from the given file
    spec = importlib.util.spec_from_file_location(function_name, filename_original)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    func = getattr(module, function_name)

    # save results in a dictionary
    results = {"passed": 0, "failed": 0, "errors": []}
    
    # run each test case
    for idx, case in enumerate(test_cases):
        try:
            # get inputs & expected output
            inputs = case["input"]
            expected = case["expected"]
            result = func(*inputs) if isinstance(inputs, (list, tuple)) else func(inputs)
            
            # Convert generator to list if needed
            if isgenerator(result):
                result = list(result)
            
            # check if result matches expected output 
            assert result == expected, f"input={inputs}, expected={expected}, got={result}"
            print(f"Test {idx+1} passed.")
            results["passed"] += 1
        except Exception as e:
            # failed test case
            print(f"Test {idx+1} failed: {e}")
            results["failed"] += 1
            results["errors"].append(f"Test {idx+1}: {str(e)}")
    
    return results

def save_refined_code(refined_code_str: str, output_filename: str = "refined_code.py"):
    """Save refined code string to a Python file."""
    # write the refined code to the specified output file
    with open(output_filename, 'w') as f:
        f.write(refined_code_str)
    print(f"Refined code saved to {output_filename}")
    return output_filename

def test_refined_function(refined_code_str: str, function_name: str, output_filename: str = None):
    """Save refined code and test it automatically using the correct test file."""
    # set default output filename if not provided
    if output_filename is None:
        output_filename = f"{function_name}_refined.py"
    
    # Check if test file exists for this function
    if function_name not in FUNCTION_TEST_FILES:
        print(f"Warning: No test file found for function '{function_name}'")
        print(f"Available functions: {list(FUNCTION_TEST_FILES.keys())}")
        return {"passed": 0, "failed": 0, "errors": [f"Function '{function_name}' not in test configuration"]}
    
    # get the test file path
    json_test_path = FUNCTION_TEST_FILES[function_name]
    
    # Save the refined code
    saved_file = save_refined_code(refined_code_str, output_filename)
    
    # Run tests and return results
    print(f"\nTesting {function_name} with {json_test_path}:")
    print("-" * 50)
    return run_tests(saved_file, function_name, json_test_path)

def test_multiple_functions(function_tests: dict):
    """Test multiple functions at once. Takes dict of {function_name: test_file}."""
    all_results = {}
    
    # iterate through each function and its test file
    for function_name, json_test_path in function_tests.items():
        print(f"\n{'='*60}")
        print(f"Testing {function_name}")
        print(f"{'='*60}")
        
        # Try to find the file - could be function_name.py or function_name_refined.py
        possible_files = [f"{function_name}.py", f"{function_name}_refined.py", "example_llm_code.py"]
        found_file = None
        
        # check for existence of possible files
        for filename in possible_files:
            try:
                with open(filename, 'r'):
                    found_file = filename
                    break
            except FileNotFoundError:
                continue
        
        # if a file was found, run tests
        if found_file:
            results = run_tests(found_file, function_name, json_test_path)
            all_results[function_name] = results
        else: # no file found
            print(f"Warning: Could not find a file containing {function_name}")
            print(f"  Tried: {possible_files}")
            all_results[function_name] = {"passed": 0, "failed": 0, "errors": ["File not found"]}
    
    # Print summary
    print(f"\n{'='*60}")
    print("SUMMARY")
    print(f"{'='*60}")
    for func_name, results in all_results.items():
        passed = results.get("passed", 0)
        failed = results.get("failed", 0)
        total = passed + failed
        print(f"{func_name}: {passed}/{total} tests passed")
    
    return all_results

# Example usage: Test refined code from Part 2
if 'refined' in globals():
    test_refined_function(refined, "bucketsort")

Refined code saved to bucketsort_refined.py

Testing bucketsort with test_case_bucketsort.json:
--------------------------------------------------
Test 1 passed.
Test 2 passed.
Test 3 passed.
Test 4 passed.
Test 5 passed.
Test 6 passed.


In [ ]:

# PART 4: Implement the Gradio Interface to run the app - Angie & Emily

import gradio as gr
import difflib

def process_code_image(image):
    """Calls any previously implemented functions established in previous code cells for the Gradio UI: transcribe, analyze, test, etc."""
    try:
        # if nothing is uploaded
        if image is None:
            return "Please upload a handwritten Python code image.", "", "", ""
        
        # try opening the image
        try:
            image = Image.open(image)
        except Exception as e:
            return "Unsupported file type. Please try again.", "", "", ""
        
        try:
            # transcribes code from uploaded image using the previously implemented function
            transcribed_code = transcribe_image_code(image)

            # check if transcribed code is empty
            if not transcribed_code.strip():
                return "Error: Transcribed code is empty.", "", "", ""

            # get name of the function from transcribed code
            match = re.search(r"def\s+(\w+)\s*\(", transcribed_code)
            if match:
                name = match.group(1)
                # auto-correct function name
                possible_names = ["flatten", "bucketsort", "pascal", "find_in_sorted", "possible_change"]
                closest = difflib.get_close_matches(name, possible_names, n=1, cutoff=0.7)
                if closest:
                    corrected_name = closest[0]
                    if corrected_name != name:
                        transcribed_code = transcribed_code.replace(f"def {name}(", f"def {corrected_name}(")
                        name = corrected_name
            else:
                return "Error: Could not detect a function name in the transcribed code.", "", "", ""
        except Exception as e:
            return f"Error during transcription: {e}", "", "", ""
        
        try:
            # analyzes and refines the transcribed code
            analysis, refined_code = analyze_code(transcribed_code) 

            # if refined code is empty, fallback to transcribed code
            if not refined_code.strip():
                refined_code = transcribed_code  # fallback

        except Exception as e:
            return transcribed_code, f"Error during analysis: {e}", "", ""

        try:
            # get func name
            match = re.search(r"def\s+(\w+)\s*\(", refined_code)
            if match:
                name = match.group(1)
            else:
                return transcribed_code, analysis, refined_code, "Error: Could not detect a function name in the refined code."

            # runs automated tests on code
            test_results = test_refined_function(refined_code, name)
            summary = f"Passed: {test_results['passed']} | Failed: {test_results['failed']}"
            if test_results["errors"]:
                summary += "\nErrors:\n" + "\n".join(test_results["errors"])
            return transcribed_code, analysis, refined_code, summary
        except Exception as e:
            return transcribed_code, analysis, refined_code, f"Error during testing: {e}"
        
    except Exception as e: # final catch-all
        return f"Unexpected error: {e}", "", "", ""

# creates Gradio app layout
with gr.Blocks(title="Code Whiteboard Tutor") as app:
    gr.Markdown("## Code Whiteboard Tutor\nUpload your handwritten Python code to get transcription, analysis, and improvements!")

    # handles inputs and output formatting/UI sections
    image_input = gr.File(type="filepath", label="Upload Handwritten Code", file_types=[".png", ".jpg", ".jpeg", ".bmp"]) #restricting to image file types only
    transcribed_output = gr.Code(label="Transcribed Code")
    analysis_output = gr.Textbox(label="Static Analysis", lines=8)
    refined_output = gr.Code(label="Refined Code")
    test_output = gr.Textbox(label = "Test Results", lines=6)

    # creates a button that will trigger processing of code snippet 
    process_btn=gr.Button("Analyze & Improve Code")
    process_btn.click(
        fn=process_code_image,
        inputs=image_input,
        outputs=[transcribed_output, analysis_output, refined_output,test_output]
    )

# launches app
if __name__ == "__main__":
    app.launch(share=True)


* Running on local URL:  http://127.0.0.1:7899
* Running on public URL: https://5822da7a5e12aa666d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Python3.12\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python3.12\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python3.12\Lib\site-packages\fastapi\applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Python3.12\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Python3.12\Lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c:\Python3.12\Lib\site-packages\starlette\middleware\errors.py", line 164, in __call__
    await self.app(scope,

Available functions: ['bucketsort', 'flatten', 'find_in_sorted', 'pascal', 'possible_change']


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Python3.12\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python3.12\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python3.12\Lib\site-packages\fastapi\applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Python3.12\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Python3.12\Lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c:\Python3.12\Lib\site-packages\starlette\middleware\errors.py", line 164, in __call__
    await self.app(scope,

Refined code saved to flatten_refined.py

Testing flatten with test_case_flatten.json:
--------------------------------------------------
Test 1 passed.
Test 2 passed.
Test 3 passed.
Test 4 passed.
Test 5 passed.
Test 6 passed.
Test 7 passed.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Python3.12\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python3.12\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python3.12\Lib\site-packages\fastapi\applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Python3.12\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Python3.12\Lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c:\Python3.12\Lib\site-packages\starlette\middleware\errors.py", line 164, in __call__
    await self.app(scope,